In [1]:
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
  

In [2]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import glob
import pandas as pd

In [5]:
path_folder = "/content/drive/.shortcut-targets-by-id/1nMf4IVSJE73nIq84zw9j3o3qGsyQeGCv/CS 245 Project/MaskMandate_Data"
os.chdir(path_folder)

In [6]:
# path = 'c:\\'
extension = 'csv'
os.chdir(path_folder)
result = glob.glob('*.{}'.format(extension))
print(result)
print(os.path.join(path_folder, result[0]))
csv_path = os.path.join(path_folder, result[0])


['MaskMandateEffects.csv']
/content/drive/.shortcut-targets-by-id/1nMf4IVSJE73nIq84zw9j3o3qGsyQeGCv/CS 245 Project/MaskMandate_Data/MaskMandateEffects.csv


In [7]:
df = pd.read_csv(csv_path)
df = df.fillna(0)
# print(df)
# 1/12/2021 - 11/20/2021
df.head()

,State,Enforced,Lifted,Enforced2,Lifted2
0,Alabama,2020-07-16,2021-04-09,0,0
1,Alaska,0,0,0,0
2,Arizona,0,0,0,0
3,Arkansas,2020-07-20,2021-03-20,0,0
4,California,2020-06-18,TBD,0,0


In [8]:
import datetime

start = datetime.datetime.strptime("2020-01-12", "%Y-%m-%d").date()
end = datetime.datetime.strptime("2021-11-20", "%Y-%m-%d").date()
df['Lifted'] = df['Lifted'].replace('TBD', end)
df['Lifted2'] = df['Lifted2'].replace('TBD', end)
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
dates =[]

for date in date_generated:
  dates.append(date.strftime("%Y-%m-%d"))
    # print(date.strftime("%m-%d-%Y"))

In [9]:
df.head()

,State,Enforced,Lifted,Enforced2,Lifted2
0,Alabama,2020-07-16,2021-04-09,0,0
1,Alaska,0,0,0,0
2,Arizona,0,0,0,0
3,Arkansas,2020-07-20,2021-03-20,0,0
4,California,2020-06-18,2021-11-20,0,0


In [10]:
full_states_col = []
for i in df.State:
  for j in dates:
    full_states_col.append((j,i))

In [11]:
print("Each date is with a state: ",len(full_states_col)==(50*len(dates)))

Each date is with a state:  True


In [34]:
def calc_date_range_enforced(beg, end, state, enf,  c = 'left'):
  # Use closed='left' to exclude end if it falls on the boundary. start- (end-1)
  dates = pd.date_range(beg,end, closed = c).strftime('%Y-%m-%d').tolist()
  if(enf==1):
    enf_list = [1]* len(dates)
  else:
    enf_list = [0]* len(dates)

  state_list = [state] * len(dates)

  return(list(zip(dates, state_list, enf_list)))

In [64]:
mandate_times = []
for index, row in df.iterrows():
  # pdb.set_trace()
  # pdb.set_trace()
  # If we have breaks between mandate enforcements
  if(row.Enforced2!=0):
    final_mask_dates = calc_date_range_enforced(start, row.Enforced, row.State, 0) + calc_date_range_enforced(row.Enforced, row.Lifted,row.State, 1) + calc_date_range_enforced(row.Lifted,row.Enforced2,row.State, 0) + calc_date_range_enforced(row.Enforced2, row.Lifted2,row.State, 1, c = None)
  # If no mask mandate at all
  elif row.Enforced ==0 and row.Lifted==0:
    final_mask_dates = calc_date_range_enforced(start, end,row.State, 0, c =None)
  # If mask mandate hasn't been lifted
  elif row.Lifted == end.strftime('%Y-%m-%d'):
    final_mask_dates = calc_date_range_enforced(start, row.Enforced,row.State, 0) + calc_date_range_enforced(row.Enforced, end,row.State, 1, c = None)
  else:
    final_mask_dates = calc_date_range_enforced(start, row.Enforced,row.State, 0) + calc_date_range_enforced(row.Enforced, row.Lifted,row.State, 1) + calc_date_range_enforced(row.Lifted, end, row.State,0, c = None)
  # pdb.set_trace()
  mandate_times.extend(final_mask_dates)

In [66]:
mandate_df = pd.DataFrame(mandate_times, columns = ['date', 'state', 'mask_mandate'])

In [69]:
mandate_df.to_csv('mask_mandate_data.csv')

In [70]:
ls -l

total 950
-rw------- 1 root root  15687 Dec  5 06:28 Create_MaskMandateData.ipynb
-rw------- 1 root root 954453 Dec  5 06:28 mask_mandate_data.csv
-rw------- 1 root root   1498 Dec  5 05:06 MaskMandateEffects.csv
-rw------- 1 root root    151 Dec  5 05:05 MaskMandateEffects.gsheet
